# unique CID quantificaton
quantifying unique CIDs to reproduce the following paper: Helal, Kazi Yasin, et al. "Public domain HTS fingerprints: design and evaluation of compound bioactivity profiles from PubChem’s bioassay repository." Journal of chemical information and modeling 56.2 (2016): 390-398.

"First, we determined 329 019 compounds that had been tested in at least 250 assays and were kept for our analysis. Second, we identified 284 assays in which at least 288 000 of these retained compounds had been tested. All other assays that had tested fewer compounds were discarded. We then removed 19 counter-screens and 22 assays that did not report primary, single concentration activity values so that, in the end, our final fingerprint assay panel consisted of __243 different assays__ (see Supplementary Data Set S1 for PubChem assay identifiers (AIDs) and descriptions). Molecule standardization identified __redundancies for 244 CIDs__, i.e. the InChI Key that they mapped to was also found for another CID. After CID merging, __328 893 unique molecules__ remained for which PubChem HTSFPs were calculated."

---
## downloading pubchem bioassay files
* search was done on pubchem bioassay (https://www.ncbi.nlm.nih.gov/pcassay) by using the __Limits__ search function, as specified in the paper (note: note: they said they got 579 assays, I got 582 - I messed around with the upload date but couldn't get the files below 582 even if I went a year before the publication, so I just downloaded the 3x extra files)
> “TotalSidCount from 10,000”, “Chemical”, “Primary Screening”, and “NIH Molecular Libraries Program"
* used _PubChem Assay Download service_ (https://pubchem.ncbi.nlm.nih.gov/assay/assaydownload.cgi) to download compressed data tables
* used 7-zip to extract .csv.gz files to .csv files and saved them to the /1-raw/public-domain-fingerprints/ dir

---
## imports

In [ ]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

## isolate all CIDs that are in >250 assays in the original 582 assays downloaded

In [ ]:
# set up imports and initialize dataframe to hold data
p = '../data/1-raw/public-domain-fingerprints/'
aid_list = os.listdir(p)

cids = pd.DataFrame(columns=['PUBCHEM_CID', 'assay_count'])
aid_count = 1

# loop over data to import, process, and add to the cids DF
for f in aid_list:
    # import and skip over desc rows
    aid = pd.read_csv(p + f, skiprows=[1,2,3])
    
    # correct CID data type, isolate unique CIDs
    cid_series = aid['PUBCHEM_CID'].dropna()
    inter_dict = {'PUBCHEM_CID':cid_series.astype(int).unique(),
              'current_count':1}
    intermediate_CID = pd.DataFrame(inter_dict)
    
    # merge with cids DF
    cids = pd.merge(cids, intermediate_CID, on='PUBCHEM_CID', how='outer')

    # add records and clean merged df
    cids = cids.fillna(0)
    cids['assay_count'] = cids['assay_count'] + cids['current_count']
    cids = cids.drop('current_count', axis=1)
    
    if aid_count%25 == 0:
        print('{count} of {total} AIDs processed'.format(count=aid_count,
                                                         total=len(aid_list)))
    aid_count += 1

cids

In [ ]:
sns.distplot(cids['assay_count'], kde=False)

In [ ]:
cids.sort_values('assay_count', ascending=False)

In [ ]:
threshold_cids = cids.loc[cids['assay_count'] > 250]
threshold_cids

In [ ]:
sp = '../data/0-meta/'
threshold_cids = threshold_cids.set_index('PUBCHEM_CID')
threshold_cids.to_csv(sp + 'KYHelal-2016-JCIM_unique_cids_thresholded.csv')

the ideal number is 329,019 compounds from the paper, but given the extra 3x screens, the couple hundred compound difference might be accounted for - given the fact that this is a fun project, this is close enough

## download SMILES from CIDs
use Entrez History Server and PUG-REST to download SMILES from CIDs

POST example:
> https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchi/cids/JSON<br>
with “Content-Type: application/x-www-form-urlencoded” in the request header, and put the string<br>
inchi=InChI=1S/C3H8/c1-3-2/h3H2,1-2H3<br>
in the POST body. (With InChI this looks a little weird, because the first “inchi=” is the name of the PUG REST argument, and the second “InChI=” is part of the InChI string itself.) You should get back CID 6334 (propane)

Entrez History Server - PUG-REST workflow example<br>
Script #3<br>
https://pubchemdocs.ncbi.nlm.nih.gov/pug-rest$example_scripts

In [ ]:
# imports
import requests
import xml.etree.ElementTree as ET
import io

sp = '../data/0-meta/'
dl_cids = pd.read_csv(sp + 'KYHelal-2016-JCIM_unique_cids_thresholded.csv')
dl_cids

In [ ]:
# set it so it loops over 10,000 compounds at a time and concats onto a final df

# set parameters for iterating through data and concatenating data
complete_id_list = dl_cids['PUBCHEM_CID'].values
id_len = len(complete_id_list)

cpnd_start = 0
batch_size = 10000
cpnd_end = cpnd_start + batch_size

smiles = pd.DataFrame()

while cpnd_start < id_len:
    # add compound batch to Entrez History Server
    db = 'pccompound'
    id_list = complete_id_list[cpnd_start:cpnd_end]

    id_string = ''
    for s in id_list:
        id_string = id_string + str(s) + ','
    id_string = id_string[:-1]

    base = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
    url = base + "epost.fcgi?db={db}".format(db = db)

    r = requests.post(url, data='id=' + id_string)
    content = r.content

    root = ET.fromstring(content)
    key = root.find('QueryKey').text
    web_env = root.find('WebEnv').text

    # convert Entrez history into a listkey for PUG-REST
    action = "entrez_to_pug"
    lg_cgi = "https://pubchem.ncbi.nlm.nih.gov/list_gateway/list_gateway.cgi?"
    lg_url = lg_cgi + 'action={a}&entrez_db={db}&entrez_query_key={key}&entrez_webenv={webenv}'.format(
        db=db,
        key=key,
        webenv=web_env,
        a=action
    )
    
    lg_result = requests.get(lg_url).content
    root = ET.fromstring(lg_result)
    list_size = root.find('Response_list-size').text
    listkey = root.find('Response_pug-listkey').text
    
    # get files via PUG-REST
    pugrest = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/"
    input_listkey = "compound/listkey/{}/".format(listkey)
    operation ='Property/IsomericSMILES/'
    output = 'CSV'

    pug_url = pugrest + input_listkey + operation + output
    urlData = requests.get(pug_url).content
    
    # open as in-memory text stream and read into a df
    temp_smiles = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
    smiles = pd.concat([smiles, temp_smiles], sort=False)
    
    print('{count} compounds of {total} total compounds converted'.format(count=cpnd_end,
                                                                          total=id_len))
    cpnd_start += batch_size
    cpnd_end = cpnd_start + batch_size
    

In [ ]:
smiles = smiles.reset_index(drop=True)

In [ ]:
sp = '../data/0-meta/'
smiles = smiles.set_index('CID')
smiles.to_csv(sp + 'KYHelal-2016-JCIM_unique_cids_SMILES.csv')

## standardize molecules by removing salt and charges and standardizing steroechemistry